In [1]:
# define and move to dataset directory
dataset_dir = './datasets'
import os
os.listdir(dataset_dir)

['dry_asphalt_severe',
 'dry_asphalt_slight',
 'dry_asphalt_smooth',
 'dry_concrete_severe',
 'dry_concrete_slight',
 'dry_concrete_smooth',
 'water_asphalt_severe',
 'water_asphalt_slight',
 'water_asphalt_smooth',
 'water_concrete_severe',
 'water_concrete_slight',
 'water_concrete_smooth',
 'wet_asphalt_severe',
 'wet_asphalt_slight',
 'wet_asphalt_smooth',
 'wet_concrete_severe',
 'wet_concrete_slight',
 'wet_concrete_smooth']

In [2]:
LABELS = ('dry_asphalt_severe',
                'dry_asphalt_slight',
                'dry_asphalt_smooth',
                'dry_concrete_severe',
                'dry_concrete_slight',
                'dry_concrete_smooth',
                'water_asphalt_severe',
                'water_asphalt_slight',
                'water_asphalt_smooth',
                'water_concrete_severe',
                'water_concrete_slight',
                'water_concrete_smooth',
                'wet_asphalt_severe',
                'wet_asphalt_slight',
                'wet_asphalt_smooth',
                'wet_concrete_severe',
                'wet_concrete_slight',
                'wet_concrete_smooth')

In [9]:
from keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 30

def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        './datasets-resize',
        target_size = (height, width), 
        classes = LABELS,
        batch_size = BATCH_SIZE,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        './datasets-resize',
        target_size = (height, width), 
        classes = LABELS,
        batch_size = BATCH_SIZE,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [4]:
import matplotlib.pyplot as plt

def plot_history(history, yrange):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    
    plt.show()

In [10]:
import keras

resnet50 = keras.applications.resnet50

train_dataset, val_dataset = generators((224,224), preprocessing=resnet50.preprocess_input)

Found 16200 images belonging to 18 classes.
Found 1800 images belonging to 18 classes.


In [6]:
conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in conv_model.layers:
    layer.trainable = False
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
predictions = keras.layers.Dense(2, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [11]:
full_model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adamax(learning_rate=0.001),
                  metrics=['acc'])
history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=10,
    epochs=5,
)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/binary_crossentropy/logistic_loss/mul/BroadcastGradientArgs defined at (most recent call last):
  File "c:\Users\levan\anaconda3\envs\rts\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\levan\anaconda3\envs\rts\lib\runpy.py", line 86, in _run_code

  File "c:\Users\levan\anaconda3\envs\rts\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\levan\anaconda3\envs\rts\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "c:\Users\levan\anaconda3\envs\rts\lib\site-packages\ipykernel\kernelapp.py", line 737, in start

  File "c:\Users\levan\anaconda3\envs\rts\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\levan\anaconda3\envs\rts\lib\asyncio\base_events.py", line 603, in run_forever

  File "c:\Users\levan\anaconda3\envs\rts\lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Users\